In [1]:
### Library Import Initialization

import numpy as np
import math
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

In [6]:
### Function to Import Stock Tickers

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
stock_data = import_stock_data(tickers, '2018-01-01')
stock_data

[*********************100%%**********************]  1 of 1 completed


,GOOG
Date,
2018-01-02,53.250000
2018-01-03,54.124001
2018-01-04,54.320000
2018-01-05,55.111500
2018-01-08,55.347000
...,...
2024-02-26,138.750000
2024-02-27,140.100006
2024-02-28,137.429993


In [10]:
### Sigma Calculation 

def compute_sigma(data):
    # Compute the standard deviation of returns
    sigma = np.std(data) / 100
    return sigma

sigma = compute_sigma(stock_data)
sigma


GOOG    0.325857
dtype: float64

In [5]:
### Function to Compute the Binomial Expected Value 

def binomial_model(S, K, T, r, sigma, option_type = 'call', steps = 1000):
    """
    Calculates the option price using the binomial model.

    Parameters:
    S (float): Current stock price
    K (float): Strike price of the option
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of the underlying stock
    option_type (str): Type of option ('call' or 'put')
    steps (int): Number of steps in the binomial tree

    Returns:
    float: Option price
    """
    # Compute Delta T
    



SyntaxError: unexpected EOF while parsing (4286946743.py, line 3)